In [1]:
from __future__ import print_function

In [2]:
def data_string_to_float(number_string):
    """
    Cette fonction convertit une chaîne de caractères représentant un nombre en une valeur flottante.
    Elle gère plusieurs cas particuliers :
        - Certaines chaînes indiquent des valeurs en millions ("M"), milliards ("B") ou milliers ("K")
          et ces unités sont converties en leur équivalent numérique.
        - Si la chaîne contient "N/A" ou "NaN", la fonction retourne "N/A" pour indiquer une valeur non disponible.
        - Si la chaîne est ">0", ce qui est une anomalie de parsing indiquant une valeur proche de zéro,
          la fonction retourne 0.
        - Pour les autres cas, la chaîne est directement convertie en float.
    :param number_string: chaîne de caractères issue d'une expression régulière à convertir en float.
    :return: représentation flottante de la chaîne, en tenant compte des signes, unités et cas particuliers.
    """
    # Si la chaîne indique une valeur non disponible, retourne "N/A"
    if ("N/A" in number_string) or ("NaN" in number_string):
        return "N/A"
    # Cas particulier : si la chaîne est ">0", on considère qu'il s'agit d'une valeur très proche de zéro
    elif number_string == ">0":
        return 0
    # Cas où la chaîne contient "B" pour milliards : on enlève le "B" et multiplie par 1 milliard
    elif "B" in number_string:
        return float(number_string.replace("B", "")) * 1000000000
    # Cas où la chaîne contient "M" pour millions : on enlève le "M" et multiplie par 1 million
    elif "M" in number_string:
        return float(number_string.replace("M", "")) * 1000000
    # Cas où la chaîne contient "K" pour milliers : on enlève le "K" et multiplie par 1 000
    elif "K" in number_string:
        return float(number_string.replace("K", "")) * 1000
    # Pour les autres chaînes, conversion directe en float
    else:
        return float(number_string)

In [3]:
def duplicate_error_check(df):
    """
    Cette fonction vérifie s'il y a des doublons dans les lignes d'un DataFrame.
    Les doublons sont souvent symptomatiques d'une erreur de parsing.
    Certains colonnes qui contiennent naturellement des valeurs identiques (comme "Unix", "Price", etc.)
    sont d'abord supprimées pour éviter de fausses alertes.
    Ensuite, pour chaque ligne, on compare chaque élément à son précédent dans la ligne et si des doublons
    sont trouvés (autres que zéro qui est fréquent), on affiche l'indice de la ligne, la ligne concernée et
    les valeurs dupliquées.
    
    :param df: le DataFrame à vérifier.
    :return: Affiche la liste des lignes contenant des doublons et les valeurs dupliquées.
    """
    # Suppression des colonnes qui peuvent contenir naturellement des doublons pour éviter des faux positifs
    df.drop(
        [
            "Unix",
            "Price",
            "stock_p_change",
            "SP500",
            "SP500_p_change",
            "Float",
            "200-Day Moving Average",
            "Short Ratio",
            "Operating Margin",
        ],
        axis=1,
        inplace=True,
    )

    # Parcours de chaque ligne du DataFrame
    for i in range(len(df)):
        # Vérification s'il existe des doublons dans la ligne
        if pd.Series(df.iloc[i] == df.iloc[i].shift()).any():
            # Récupération des valeurs apparaissant plus d'une fois dans la ligne
            duplicates = set(
                [x for x in list(df.iloc[i]) if list(df.iloc[i]).count(x) > 1]
            )
            # On ignore le cas fréquent où la valeur 0 est dupliquée
            if duplicates != {0}:
                # Affichage de l'indice de la ligne, de la ligne elle-même et des valeurs dupliquées
                print(i, df.iloc[i], duplicates, sep = "\n")

In [4]:
def status_calc(stock, sp500, outperformance=10):
    """
    Cette fonction détermine si une action (stock) a surperformé l'indice S&P500 d'un certain seuil.
    Le paramètre outperformance représente le nombre minimal de points par lesquels l'action doit dépasser
    le S&P500 pour être considérée comme surperformante.
    
    :param stock: la valeur de l'action.
    :param sp500: la valeur de l'indice S&P500.
    :param outperformance: seuil de surperformance (doit être positif). Par défaut, 10.
    :return: True si l'action surperforme le S&P500 de la valeur du seuil, False sinon.
    """
    # Vérifie que le seuil de surperformance est positif, sinon lève une erreur
    if outperformance < 0:
        raise ValueError("outperformance must be positive")
    # Calcul de la différence entre le stock et le S&P500 et comparaison avec le seuil
    return stock - sp500 >= outperformance